## ASSUMING P/E RATIOS

In [15]:
import pandas as pd
import numpy as np
import json
import glob

csv_files = glob.glob('csvs/*.csv')

# Assuming the following PE Ratios for the stocks
pe_ratios = {
    'AAPL': 30,
    'AMZN': 80,
    'BA': 25,
    'CHGG': 60,
    'MSFT': 35,
    'TSLA': 100
}

stock_data = []

for file in csv_files:
    stock_ticker = file.split('/')[-1].split('.')[0] 
    df = pd.read_csv(file)
    df['Stock'] = stock_ticker
    df['PE_Ratio'] = pe_ratios.get(stock_ticker, np.nan)
    stock_data.append(df)

data = pd.concat(stock_data)

undervalued_stocks = data[data['PE_Ratio'] <= 50]

undervalued_stocks['Price_Growth'] = undervalued_stocks.groupby('Stock')['Close'].pct_change(periods=252)

top_stocks = undervalued_stocks.groupby('Stock').agg({'Price_Growth': 'max'}).reset_index()
top_stocks = top_stocks.sort_values(by='Price_Growth', ascending=False).head(3)

budget = 10000
if len(top_stocks) > 0:
    investment_per_stock = budget / len(top_stocks)

    strategy = {}
    for _, row in top_stocks.iterrows():
        stock = row['Stock']
        strategy[stock] = {
            "Amount Invested": f"${investment_per_stock:.2f}",
            "Day to Sell": (pd.to_datetime(data['Date'].max()) + pd.Timedelta(days=10)).strftime('%Y-%m-%d')
        }

    strategy_json = json.dumps(strategy, indent=4)
    print(strategy_json)
else:
    print("No undervalued stocks with positive price growth found.")


No undervalued stocks with positive price growth found.


## IGNORING P/E RATIOS

In [11]:
import pandas as pd
import numpy as np
import json
import glob

csv_files = glob.glob('csvs/*.csv')

stock_data = []

for file in csv_files:
    stock_ticker = file.split('/')[-1].split('.')[0]
    df = pd.read_csv(file)
    df['Stock'] = stock_ticker
    stock_data.append(df)

data = pd.concat(stock_data)

data['Price_Growth'] = data.groupby('Stock')['Close'].pct_change(periods=252)

top_stocks = data.groupby('Stock').agg({'Price_Growth': 'max'}).reset_index()
top_stocks = top_stocks.sort_values(by='Price_Growth', ascending=False).head(3)

budget = 10000
if len(top_stocks) > 0:
    investment_per_stock = budget / len(top_stocks)

    strategy = {}
    for _, row in top_stocks.iterrows():
        stock = row['Stock']
        strategy[stock] = {
            "Amount Invested": f"${investment_per_stock:.2f}",
            "Day to Sell": (pd.to_datetime(data['Date'].max()) + pd.Timedelta(days=10)).strftime('%Y-%m-%d')
        }

    strategy_json = json.dumps(strategy, indent=4)
    print(strategy_json)
else:
    print("No stocks with positive price growth found.")

{
    "csvs\\AAPL": {
        "Amount Invested": "$3333.33",
        "Day to Sell": "2024-06-13"
    },
    "csvs\\AMZN": {
        "Amount Invested": "$3333.33",
        "Day to Sell": "2024-06-13"
    },
    "csvs\\BA": {
        "Amount Invested": "$3333.33",
        "Day to Sell": "2024-06-13"
    }
}
